In [0]:
##############################################################################

###### Producer of Financial Data Stream morphed into Meteo Data Stream ######

##############################################################################


In [0]:
import socket
import time
import pandas as pd
from datetime import datetime
from pyspark import SparkFiles

### Upload the csv-files from the external website ###
### Import 12 files for 12 months 2022

url1 = "https://www.web.statistik.zh.ch/awel/LoRa/data/AWEL_Sensors_LoRa_202201.csv"
url2 = "https://www.web.statistik.zh.ch/awel/LoRa/data/AWEL_Sensors_LoRa_202202.csv"
url3 = "https://www.web.statistik.zh.ch/awel/LoRa/data/AWEL_Sensors_LoRa_202203.csv"
url4 = "https://www.web.statistik.zh.ch/awel/LoRa/data/AWEL_Sensors_LoRa_202204.csv"
url5 = "https://www.web.statistik.zh.ch/awel/LoRa/data/AWEL_Sensors_LoRa_202205.csv"
url6 = "https://www.web.statistik.zh.ch/awel/LoRa/data/AWEL_Sensors_LoRa_202206.csv"
url7 = "https://www.web.statistik.zh.ch/awel/LoRa/data/AWEL_Sensors_LoRa_202207.csv"
url8 = "https://www.web.statistik.zh.ch/awel/LoRa/data/AWEL_Sensors_LoRa_202208.csv"
url9 = "https://www.web.statistik.zh.ch/awel/LoRa/data/AWEL_Sensors_LoRa_202209.csv"
url10 = "https://www.web.statistik.zh.ch/awel/LoRa/data/AWEL_Sensors_LoRa_202210.csv"
url11 = "https://www.web.statistik.zh.ch/awel/LoRa/data/AWEL_Sensors_LoRa_202211.csv"
url12 = "https://www.web.statistik.zh.ch/awel/LoRa/data/AWEL_Sensors_LoRa_202212.csv"

spark.sparkContext.addFile(url1)
inputFile1 = spark.read.csv("file://"+SparkFiles.get("AWEL_Sensors_LoRa_202201.csv"), header=True, inferSchema=True, sep=';')

spark.sparkContext.addFile(url2)
inputFile2 = spark.read.csv("file://"+SparkFiles.get("AWEL_Sensors_LoRa_202202.csv"), header=True, inferSchema=True, sep=';')

spark.sparkContext.addFile(url3)
inputFile3 = spark.read.csv("file://"+SparkFiles.get("AWEL_Sensors_LoRa_202203.csv"), header=True, inferSchema=True, sep=';')

spark.sparkContext.addFile(url4)
inputFile4 = spark.read.csv("file://"+SparkFiles.get("AWEL_Sensors_LoRa_202204.csv"), header=True, inferSchema=True, sep=';')

spark.sparkContext.addFile(url5)
inputFile5 = spark.read.csv("file://"+SparkFiles.get("AWEL_Sensors_LoRa_202205.csv"), header=True, inferSchema=True, sep=';')

spark.sparkContext.addFile(url6)
inputFile6 = spark.read.csv("file://"+SparkFiles.get("AWEL_Sensors_LoRa_202206.csv"), header=True, inferSchema=True, sep=';')

spark.sparkContext.addFile(url7)
inputFile7 = spark.read.csv("file://"+SparkFiles.get("AWEL_Sensors_LoRa_202207.csv"), header=True, inferSchema=True, sep=';')

spark.sparkContext.addFile(url8)
inputFile8 = spark.read.csv("file://"+SparkFiles.get("AWEL_Sensors_LoRa_202208.csv"), header=True, inferSchema=True, sep=';')

spark.sparkContext.addFile(url9)
inputFile9 = spark.read.csv("file://"+SparkFiles.get("AWEL_Sensors_LoRa_202209.csv"), header=True, inferSchema=True, sep=';')

spark.sparkContext.addFile(url10)
inputFile10 = spark.read.csv("file://"+SparkFiles.get("AWEL_Sensors_LoRa_202210.csv"), header=True, inferSchema=True, sep=';')

spark.sparkContext.addFile(url11)
inputFile11 = spark.read.csv("file://"+SparkFiles.get("AWEL_Sensors_LoRa_202211.csv"), header=True, inferSchema=True, sep=';')

spark.sparkContext.addFile(url12)
inputFile12 = spark.read.csv("file://"+SparkFiles.get("AWEL_Sensors_LoRa_202212.csv"), header=True, inferSchema=True, sep=';')
 
### Combine the 12 singel dataframes into one single one

Mega_inputFile = pd.concat([inputFile1, inputFile2, inputFile3, inputFile4, inputFile5, inputFile6, inputFile7, inputFile8, inputFile9, inputFile10, inputFile11, inputFile12], ignore_index = True, sort = False)

### This function is needed for the transformation of the files into Pandas data frames

spark.conf.set("spark.sql.execcution.arrow.enabled","true")

### Start the sending out of the extended dataFrame

def send():
  
  ifile = Mega_inputFile.toPandas()
  i = 0

  while True:
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    client_socket.bind(('localhost', 9998))
    client_socket.listen(10)
    conn, addr = client_socket.accept()
    try:  
      while True:
        start = time.time()
        for row in ifile.iterrows():
          print("sending: "+ row.rstrip() + "," + str(datetime.now()))
          message = row.rstrip() + ',' + str(datetime.now()) + "\n"
          message = message.encode()
          conn.send(message)
          # send data every 100 ms
          #TODO: change the input rate here (or remove) to see effects in the consumer on total number of items processed per window
          time.sleep(0.1)

    except Exception as e:
        print(str(e))
        conn.close()
        client_socket.close()
        continue
    finally:
        conn.close()
        client_socket.close()
send()